# Transformer from Attention Is All You Need

In [1]:
# import necessary packages
import torch
import torch.nn as nn 

ERROR: Could not find a version that satisfies the requirement torchtext==0.10.0 (from versions: 0.1.1, 0.2.0, 0.2.1, 0.2.3, 0.3.1, 0.4.0, 0.5.0, 0.6.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.0, 0.15.1)
ERROR: No matching distribution found for torchtext==0.10.0

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Attention

#### See section 3.2 

We'll start by building the self attention module, this is what makes the transformer architecture so powerfull. It takes in an embedding size, and a number of heads. We split the embedding into however many heads there are, this is multi-head attention. 

(ex. if we have an embedding size 256 and heads is 8, we split it into 8x32 parts)

In [107]:
class SelfAttention(nn.Module):
  def __init__(self, embed_size, heads):
    super(SelfAttention, self).__init__()

    assert (embed_size % heads == 0), 'Embed size not divisible by heads'

    self.embed_size = embed_size
    self.heads = heads
    self.head_dimension = embed_size // heads

    # define linear layers to send queries, keys, and values through
    self.values = nn.Linear(self.head_dimension, self.head_dimension, bias=False)
    self.keys = nn.Linear(self.head_dimension, self.head_dimension, bias=False)
    self.queries = nn.Linear(self.head_dimension, self.head_dimension, bias=False)
    
    # concatenate heads after multi-head attention
    self.fully_connected_out = nn.Linear(embed_size, embed_size)

  def forward(self, values, keys, query, mask):
    # num training examples to send in at one time
    N = query.shape[0]

    # these correspond to source sentence length and target sentence length
    value_len = values.shape[1]
    key_len = keys.shape[1]
    query_len = query.shape[1]

    # split embeddings into multiple heads
    values = values.reshape(N, value_len, self.heads, self.head_dimension)
    keys = keys.reshape(N, key_len, self.heads, self.head_dimension)
    queries = query.reshape(N, query_len, self.heads, self.head_dimension)

    # send through linear layers
    values = self.values(values)
    keys = self.keys(keys)
    queries = self.queries(queries)

    #------- MatMul Q and K(Transposed) ---------#
      # queries shape: (N, query_len, heads, head_dimension)
      # keys shape: (N, key_len, heads, head_dimension)
    # we want
      # QdotK shape: (N, heads, query_len, key_len)
    QdotKt = torch.einsum('nqhd,nkhd->nhqk', [queries, keys])

    #------------ Scale ------------#
    # QdotKt = QdotKt / (self.embed_size ** (1/2))

    #----- Mask (for decoder) ------#
    # decoder requires masked multi-head attention
    if mask is not None:
      # closes elements above the diagonal so that the model can't see future values
      QdotKt = QdotKt.masked_fill(mask == 0, float('-1e20'))

    #---------- Softmax ------------#
    soft = torch.softmax(QdotKt / (self.embed_size ** (1/2)), dim=3)
    # attention = torch.softmax(QdotKt)

    #------ MatMul soft and V ------#
      # soft shape: (N, heads, query_len, key_len)
      # values shape: (N, value_len, heads, head_dimension)
    # we want
      # (N, query_len, heads, head_dimension)
      # after multiplying, flatten last two dimensions
    out = torch.einsum('nhql,nlhd->nqhd', [soft,values]).reshape(
      N, query_len, self.heads*self.head_dimension)

    #------ Concatenate heads ------#
    out = self.fully_connected_out(out)
    
    return out


## Transformer Block
Now that we have out attention mechanism we can construct the transformer block, which will be used to create both the encoder and decoder.

In [108]:
class TransformerBlock(nn.Module):
  # transformer block architecture
    # attention block -> add & normalize -> feed forward -> add & normalize

  def __init__(self, embed_size, heads, dropout, forward_expansion):
    super(TransformerBlock, self).__init__()
    self.attention = SelfAttention(embed_size, heads)
    self.norm1 = nn.LayerNorm(embed_size)
    self.norm2 = nn.LayerNorm(embed_size)

    self.feed_forward = nn.Sequential(
      nn.Linear(embed_size, forward_expansion * embed_size),
      nn.ReLU(),
      nn.Linear(forward_expansion * embed_size, embed_size)
    )

    self.dropout = nn.Dropout(dropout)

  def forward(self, value, key, query, mask):
    # attention
    attention = self.attention(value, key, query, mask)

    # add & norm
    x = self.dropout(self.norm1(attention + query))

    # feed forward
    forward = self.feed_forward(x)

    # add & norm
    out = self.dropout(self.norm2(forward + x))

    return out


## Encoder

This encoder represents the left part of the diagram in Figure 1.

Because the previous components we built will be used in the encoder, we have to pass through all of the hyperparameters.

There is a new parameter called max_length, this is related to the positional enbedding. We have to tell the model what our max length of sentence is. It will vary based on the dataset.

In [109]:
class Encoder(nn.Module):
  def __init__(
    self,
    src_vocab_size,
    embed_size,
    num_layers,
    heads,
    device,
    forward_expansion,
    dropout,
    max_length
  ):

    super(Encoder, self).__init__()
    # define embeddings for input
    self.embed_size = embed_size
    self.device = device
    self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
    self.positional_embedding = nn.Embedding(max_length, embed_size)

    # define layers, which consists of a single transformer blocks
    self.layers = nn.ModuleList(
      [
        TransformerBlock(
          embed_size,
          heads,
          dropout=dropout,
          forward_expansion=forward_expansion
        )
        for _ in range(num_layers)
      ]
    )
    
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, mask):
    # we get the number of examples and the sequence legnth
    N, seq_len = x.shape

    # we create a range of 0 to the sequence length for every example
    positions = torch.arange(0, seq_len).expand(N, seq_len).to(self.device)

    # we give the word embedding and the postions and the model will now know the positions of words
    out = self.dropout(self.word_embedding(x) + self.positional_embedding(positions))

    for layer in self.layers:
      # in the encoder, all our v, k, q input vectors are the same
      out = layer(out, out, out, mask)

    return out

## Decoder

First we will create the decoder block, without the embeddings or linear and softmax which are outside.

In [110]:
class DecoderBlock(nn.Module):
  def __init__(
    self,
    embed_size,
    heads,
    forward_expansion,
    dropout,
    device
  ):
    super(DecoderBlock, self).__init__()
    
    self.attention = SelfAttention(embed_size, heads)
    self.norm = nn.LayerNorm(embed_size)
    self.transformer_block = TransformerBlock(
      embed_size, heads, dropout, forward_expansion
    )
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, value, key, src_mask, trg_mask):
    # masked attention
    attention = self.attention(x,x,x, trg_mask)

    # add and norm
    query = self.dropout(self.norm(attention + x))

    # transformer block
    out = self.transformer_block(value, key, query, src_mask)

    return out


Next we can use the decoder block make the whole decoder.

In [111]:
class Decoder(nn.Module):
  def __init__(
    self,
    trg_vocab_size,
    embed_size,
    num_layers,
    heads,
    forward_expansion,
    dropout,
    device,
    max_length
  ):
    super(Decoder, self).__init__()

    # define embeddings 
    self.device = device
    self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
    self.positional_embedding = nn.Embedding(max_length, embed_size)

    # create num_layers of the decoder block 
    self.layers = nn.ModuleList(
      [
        DecoderBlock(
          embed_size, heads, forward_expansion, dropout, device
        )
        for _ in range(num_layers)
      ]
    )

    # final linear layer
    self.fully_connected_out = nn.Linear(embed_size, trg_vocab_size)

    self.dropout = nn.Dropout(dropout)

  def forward(self, x, encoder_out, src_mask, trg_mask):
    # we get the number of examples and the sequence legnth
    N, seq_len = x.shape
    
    # we create a range of 0 to the sequence length for every example
    positions = torch.arange(0, seq_len).expand(N, seq_len).to(self.device)

    # we give the word embedding and the postions and the model will now know the positions of words
    x = self.dropout((self.word_embedding(x) + self.positional_embedding(positions)))

    # run the decoder block for all of the layers
    for layer in self.layers:
      x = layer(x, encoder_out, encoder_out, src_mask, trg_mask)

    # run final linear layer
    out = self.fully_connected_out(x)

    # final softmax 
    # out = torch.softmax(out)

    return out


## Transformer

Now that we've constructed all of the components, we can put them together.

In [112]:
class Transformer(nn.Module):
  def __init__(
    self,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    trg_pad_idx,
    embed_size=256,
    num_layers=6,
    forward_expansion=4,
    heads=8,
    dropout=0,
    # device='cuda',
    device = 'cpu',
    max_length=100
  ):
    super(Transformer, self).__init__()

    # define encoder
    self.encoder = Encoder(
      src_vocab_size,
      embed_size,
      num_layers,
      heads,
      device,
      forward_expansion,
      dropout,
      max_length
    )

    # define decoder
    self.decoder = Decoder(
      trg_vocab_size,
      embed_size,
      num_layers,
      heads,
      forward_expansion,
      dropout,
      device,
      max_length
    )

    self.src_pad_idx = src_pad_idx
    self.trg_pad_idx = trg_pad_idx
    self.device = device

  # define make src mask
  def make_src_mask(self, src):
    # we want the src_mask in the shape of (N, 1, 1, src_len)
    # if src is the src pad index then it will be 1, if not it will be 0
    src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
    return src_mask.to(self.device)

  # define make trg mask
  def make_trg_mask(self, trg):
    N, trg_len = trg.shape

    # we want a lower triangular matrix 
    trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
      N, 1, trg_len, trg_len
    )

    return trg_mask.to(self.device)

  def forward(self, src, trg):
    src_mask = self.make_src_mask(src)
    trg_mask = self.make_trg_mask(trg)

    enc_src = self.encoder(src, src_mask)

    out = self.decoder(trg, enc_src, src_mask, trg_mask)

    return out

## Testing

Now that we've created the transformer model, we can test it to see if it works.

In [113]:
if __name__ == '__main__':
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  x = torch.tensor([[1,5,6,4,3,9,5,2,0], [1,8,7,3,4,5,6,7,2]]).to(device)

  trg = torch.tensor([[1,7,4,3,5,9,2,0], [1,5,6,2,4,7,6,2]]).to(device)

  src_pad_idx = 0
  trg_pad_idx = 0
  src_vocab_size = 10
  trg_vocab_size = 10

  model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx).to(device)

  out = model(x, trg[:, :-1])

  print(out.shape)

torch.Size([2, 7, 10])
